In [1]:
%load_ext cython

In [ ]:
%%cython -a

# distutils: language = c++


from libc.stdlib cimport malloc, free
from libc.string cimport memset
from libcpp.vector cimport vector
from libcpp.algorithm cimport copy
from libcpp.pair cimport pair
from libcpp.unordered_map cimport unordered_map
import numpy as np
cimport numpy as np


cdef void find_paths(int[:, :] graph_arcs, double[:,:] graph_data,  int num_arcs, int src, int dest,
                     int[:] visited, int[:] path, int path_index,
                     double[:] resources, double[:] resource_limits,
                     int num_resources, list all_paths):
    """
    Recursive function to find all paths from src to dest with resource constraints.
    """
    cdef int i, head, tail, r
    cdef bint valid
    visited[src] = 1
    path[path_index] = src
    path_index += 1

    if src == dest:
        all_paths.append([path[i] for i in range(path_index)])
    else:
        for i in range(num_arcs):
            head = graph_arcs[i, 0]
            tail = graph_arcs[i, 1]
            if head == src and not visited[tail]:
                valid = 1
                for r in range(num_resources):
                    resources[r] += graph_data[i, r]
                    if resources[r] > resource_limits[r]:
                        valid = 0

                if valid:
                    find_paths(graph_arcs, graph_data, num_arcs, tail, dest, visited, path, path_index, resources, resource_limits, num_resources, all_paths)

                # Backtrack resource consumption
                for r in range(num_resources):
                    resources[r] -= graph_data[i, r]

    # Backtrack
    visited[src] = 0
    path_index -= 1

cdef void propage(int parent_node, vector[int]& parent_path, vector[int]& parent_reachable, vector[double]& parent_resource,
                  vector[double]& r_max, vector[vector[int]]& adj_list, double[:,:,:] graph_data, int num_resources,
                  vector[vector[int]]& paths, vector[vector[double]]& path_resources, vector[vector[int]]& paths_reachables):
    cdef int i, j, k
    cdef int child_node
    cdef vector[int] child_path
    cdef vector[double] child_resource
    cdef vector[int] child_reachables

    for i in range(adj_list[parent_node].size()):
        child_node = adj_list[parent_node][i]
        child_path = copy(parent_path)
        child_path.push_back(child_node)
        child_resource = copy(parent_resource)
        child_reachables = copy(parent_reachable)


        for j in range(num_resources):
            child_resource[j] += graph_data[parent_node, child_node, j]

        for  i in range(child_reachables.size()):
            if child_reachables[i] == child_node:
                child_reachables.erase(child_reachables.begin() + i)
            else:
                for j in range(1,num_resources):
                    if child_resource[j]+graph_data[child_node,child_reachables[i],j] > r_max[j]:
                        child_reachables.erase(child_reachables.begin() + i)
                        break
        paths.push_back(child_path)
        path_resources.push_back(child_resource)
        paths_reachables.push_back(child_reachables)
        
        

        
       
        
            
            







def generate_all_paths(int[:,:] graph_arcs, double[:, :] graph_data, int num_nodes, int src, int dest,
                       double[:] resource_limits):
    """
    Generate all possible paths in a graph from source to destination considering resource constraints.
    Returns:
        List of all paths satisfying resource constraints.
    """
    cdef int num_arcs = graph_data.shape[0]
    cdef int num_resources = len(resource_limits)
    cdef np.ndarray[int, ndim=1] visited = np.zeros(num_nodes, dtype=np.int32)
    cdef np.ndarray[int, ndim=1] path = np.zeros(num_nodes, dtype=np.int32)
    cdef np.ndarray[double, ndim=1] resources = np.zeros(num_resources, dtype=np.float64)
    cdef int path_index = 0

    cdef vector[vector[int]] paths
    cdef vector[vector[double]] path_resources
    cdef vector[vector[int]] paths_reachables

    cdef list all_paths = []
    cdef int i, j
    cdef vector[vector[int]] adj_list

    for i in range(num_nodes):
        adj_list.push_back(vector[int]())
    for i in range(num_arcs):
        adj_list[graph_arcs[i, 0]].push_back(graph_arcs[i, 1])
    

    # Call the recursive function
    find_paths(graph_arcs,graph_data, num_arcs, src, dest, visited, path, path_index, resources, resource_limits, num_resources, all_paths)

    return all_paths

Content of stdout:
_cython_magic_64be28be7cb0d765a1e78c23d2b784b0d0e64228.cpp
c:\Users\rezam\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\rezam\.ipython\cython\_cython_magic_64be28be7cb0d765a1e78c23d2b784b0d0e64228.cpp(20606): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\rezam\.ipython\cython\_cython_magic_64be28be7cb0d765a1e78c23d2b784b0d0e64228.cpp(20616): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\rezam\.ipython\cython\_cython_magic_64be28be7cb0d765a1e78c23d2b784b0d0e64228.cpp(26947): warning C4551: function call missing argument list
   Creating library C:\Users\rezam\.ipython\cython\Users\rezam\.ipython\cython\_cython_magic_64be28be7cb0d765a1e78c23d2b784b0d0e64228.cp312-win_amd64.lib and object C:\Users\reza

In [63]:
import numpy as np
import random

# Example graph data: [head, tail, weight, resource1, resource2]

graph_data_ = np.array([[i,j,random.randint(1,6), random.randint(1,5)] for i in range(10) for j in range(10) if i!=j], dtype=np.float64)

graph_arcs = graph_data_[:, :2].astype(np.int32)

graph_data = graph_data_[:, 2:].astype(np.float64)

# Define constraints
num_nodes = 10
src = 0
dest = 9
resource_limits = np.array([15.0, 13.0], dtype=np.float64)  # Constraints on resources 1 and 2

# Generate all paths
paths = generate_all_paths(graph_arcs, graph_data, num_nodes, src, dest, resource_limits)
print("Valid paths:", paths)


Valid paths: [[0, 1, 2, 4, 9], [0, 1, 2, 5, 9], [0, 1, 2, 9], [0, 1, 3, 9], [0, 1, 4, 9], [0, 1, 5, 9], [0, 1, 6, 3, 9], [0, 1, 6, 4, 9], [0, 1, 6, 9], [0, 1, 7, 3, 9], [0, 1, 7, 5, 9], [0, 1, 8, 4, 9], [0, 1, 8, 5, 9], [0, 1, 8, 6, 4, 9], [0, 1, 8, 9], [0, 1, 9], [0, 2, 1, 5, 9], [0, 2, 1, 8, 9], [0, 2, 1, 9], [0, 2, 3, 1, 9], [0, 2, 3, 5, 9], [0, 2, 3, 8, 9], [0, 2, 4, 1, 9], [0, 2, 4, 9], [0, 2, 5, 9], [0, 2, 6, 1, 9], [0, 2, 6, 4, 1, 9], [0, 2, 6, 4, 9], [0, 2, 6, 9], [0, 2, 7, 1, 9], [0, 2, 7, 4, 9], [0, 2, 7, 5, 9], [0, 2, 7, 9], [0, 2, 8, 1, 9], [0, 2, 8, 3, 9], [0, 2, 8, 5, 9], [0, 2, 8, 6, 4, 1, 9], [0, 2, 8, 6, 4, 9], [0, 2, 8, 6, 9], [0, 2, 8, 9], [0, 2, 9], [0, 3, 1, 2, 5, 9], [0, 3, 1, 2, 9], [0, 3, 1, 4, 9], [0, 3, 1, 5, 9], [0, 3, 1, 6, 4, 9], [0, 3, 1, 6, 9], [0, 3, 1, 7, 5, 9], [0, 3, 1, 8, 5, 9], [0, 3, 1, 8, 6, 4, 9], [0, 3, 1, 8, 9], [0, 3, 1, 9], [0, 3, 2, 1, 5, 9], [0, 3, 2, 1, 9], [0, 3, 2, 4, 1, 9], [0, 3, 2, 4, 9], [0, 3, 2, 5, 9], [0, 3, 2, 6, 1, 9], [0, 3, 2,

In [58]:
def find_paths_py(graph_data, num_nodes, src, dest, visited, path, path_index,
                  resources, resource_limits, num_resources, all_paths):
    """
    Recursive function to find all paths from src to dest with resource constraints in Python.
    """
    visited[src] = True
    path[path_index] = src
    path_index += 1

    if src == dest:
        all_paths.append(path[:path_index])
    else:
        for row in graph_data:
            head, tail = int(row[0]), int(row[1])
            if head == src and not visited[tail]:
                valid = True
                for r in range(num_resources):
                    resources[r] += row[2+ r]
                    if resources[r] > resource_limits[r]:
                        valid = False

                if valid:
                    find_paths_py(graph_data, num_nodes, tail, dest, visited, path, 
                                  path_index, resources, resource_limits, num_resources, all_paths)

                # Backtrack resource consumption
                for r in range(num_resources):
                    resources[r] -= row[2+ r]

    # Backtrack
    visited[src] = False
    path_index -= 1


def generate_all_paths_py(graph_data, num_nodes, src, dest, resource_limits):
    """
    Generate all possible paths in a graph from source to destination considering resource constraints.

    Arguments:
        graph_data: 2D list where each row represents [head, tail, weight, resource_1, resource_2, ...].
        num_nodes: Total number of nodes in the graph.
        src: Source node index.
        dest: Destination node index.
        resource_limits: List of resource constraints.

    Returns:
        List of all paths satisfying resource constraints.
    """
    visited = [False] * num_nodes
    path = [0] * num_nodes
    resources = [0.0] * len(resource_limits)
    all_paths = []
    path_index = 0
    num_resources = len(resource_limits)

    # Start recursion
    find_paths_py(graph_data, num_nodes, src, dest, visited, path, path_index,
                  resources, resource_limits, num_resources, all_paths)

    return all_paths


In [59]:
paths = generate_all_paths_py(graph_data_, num_nodes, src, dest, resource_limits)
print("Valid paths:", paths)

Valid paths: [[0, 1, 2, 3, 9], [0, 1, 2, 9], [0, 1, 3, 2, 9], [0, 1, 3, 4, 9], [0, 1, 3, 5, 9], [0, 1, 3, 6, 9], [0, 1, 3, 7, 6, 9], [0, 1, 3, 7, 9], [0, 1, 3, 8, 9], [0, 1, 3, 9], [0, 1, 4, 3, 9], [0, 1, 4, 5, 9], [0, 1, 4, 8, 9], [0, 1, 4, 9], [0, 1, 5, 2, 9], [0, 1, 5, 3, 9], [0, 1, 5, 4, 3, 9], [0, 1, 5, 4, 9], [0, 1, 5, 6, 9], [0, 1, 5, 7, 3, 9], [0, 1, 5, 7, 6, 9], [0, 1, 5, 7, 9], [0, 1, 5, 8, 3, 9], [0, 1, 5, 8, 9], [0, 1, 5, 9], [0, 1, 6, 9], [0, 1, 7, 6, 9], [0, 1, 8, 2, 9], [0, 1, 8, 3, 9], [0, 1, 8, 5, 9], [0, 1, 8, 9], [0, 1, 9], [0, 2, 1, 3, 9], [0, 2, 1, 5, 9], [0, 2, 1, 9], [0, 2, 3, 1, 9], [0, 2, 3, 4, 9], [0, 2, 3, 5, 9], [0, 2, 3, 7, 9], [0, 2, 3, 9], [0, 2, 5, 9], [0, 2, 6, 1, 9], [0, 2, 6, 9], [0, 2, 7, 1, 9], [0, 2, 7, 3, 9], [0, 2, 7, 5, 9], [0, 2, 7, 6, 1, 9], [0, 2, 7, 6, 4, 9], [0, 2, 7, 6, 9], [0, 2, 7, 9], [0, 2, 8, 9], [0, 2, 9], [0, 3, 1, 5, 9], [0, 3, 1, 8, 9], [0, 3, 1, 9], [0, 3, 2, 1, 5, 9], [0, 3, 2, 1, 9], [0, 3, 2, 6, 1, 9], [0, 3, 2, 6, 9], [0, 3, 

In [60]:
%timeit generate_all_paths(graph_arcs, graph_data, num_nodes, src, dest, resource_limits)


989 μs ± 85.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [61]:

%timeit generate_all_paths_py(graph_data_, num_nodes, src, dest, resource_limits)

104 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
